# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
! pip install XlsxWriter
! pip install numpy

  Obtaining dependency information for XlsxWriter from https://files.pythonhosted.org/packages/a7/ea/53d1fe468e63e092cf16e2c18d16f50c29851242f9dd12d6a66e0d7f0d02/XlsxWriter-3.2.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 3.4 MB/s eta 0:00:00a 0:00:01


In [2]:
import numpy as np
import pandas as pd
import requests 
import xlsxwriter
import math

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [4]:
stocks = pd.read_csv('sp_500_stocks.csv')


In [5]:
stocks.size
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [6]:
%load_ext autoreload
%autoreload 2

In [7]:
from secrets2 import IEX_CLOUD_API_TOKEN
IEX_CLOUD_API_TOKEN

'Tpk_059b97af715d417d9f49f50b51b1c448'

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [8]:
symbol = 'AAPL'
domain = 'https://sandbox.iexapis.com'
path = f'/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
IEX_CLOUD_API_TOKEN = "sk_750cfa6e35bd4716a767deac4c6dcb0f"

url = domain+ path
url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
print(url)
data = requests.get(url)

https://cloud.iexapis.com/stable/stock/AAPL/quote?token=sk_750cfa6e35bd4716a767deac4c6dcb0f


In [9]:
data = data.json()

In [10]:
data['latestPrice']
market_cap = data['marketCap']
market_cap
print(market_cap)


s = "{:e}".format(market_cap)
print(s)

2806903710100
2.806904e+12


## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [11]:
price = data['latestPrice']
market_cap = data['marketCap']

## Adding Our Stocks Data to a Pandas DataFrame

The next thing we need to do is add our stock's price and market capitalization to a pandas DataFrame. Think of a DataFrame like the Python version of a spreadsheet. It stores tabular data.

In [12]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy


In [13]:
final_df_1 = pd.DataFrame(['AAPL', 
                        data['latestPrice'], 
                        data['marketCap'], 
                        'N/A'], 
                        index = my_columns).T
final_df_1 

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,183.05,2806903710100,N/A


In [14]:
pd.concat([final_dataframe, final_df_1], ignore_index=True)

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,AAPL,183.05,2806903710100,N/A


## Looping Through The Tickers in Our List of Stocks

Using the same logic that we outlined above, we can pull data for all S&P 500 stocks and store their data in the DataFrame using a `for` loop.

In [92]:
stocks['Ticker'].head(50)

0        A
1      AAL
2      AAP
3     AAPL
4     ABBV
5      ABC
6     ABMD
7      ABT
8      ACN
9     ADBE
10     ADI
11     ADM
12     ADP
13    ADSK
14     AEE
15     AEP
16     AES
17     AFL
18     AIG
19     AIV
20     AIZ
21     AJG
22    AKAM
23     ALB
24    ALGN
25     ALK
26     ALL
27    ALLE
28    ALXN
29    AMAT
30    AMCR
31     AMD
32     AME
33    AMGN
34     AMP
35     AMT
36    AMZN
37    ANET
38    ANSS
39    ANTM
40     AON
41     AOS
42     APA
43     APD
44     APH
45    APTV
46     ARE
47     ATO
48    ATVI
49     AVB
Name: Ticker, dtype: object

In [93]:
my_temp_list = []


for stock in stocks['Ticker']:
    try:
        url = f'https://cloud.iexapis.com/stable/stock/{stock}/quote?token={IEX_CLOUD_API_TOKEN}'
        print(url)
        data = requests.get(url).json()
        print(data)
        #final_dataframe = pd.concat([final_dataframe, ser])
        my_temp_list.append(data)
    except:
        print(f'problem with {stock}')

df = pd.DataFrame(my_temp_list, columns=my_columns)


https://cloud.iexapis.com/stable/stock/A/quote?token=sk_750cfa6e35bd4716a767deac4c6dcb0f
{'avgTotalVolume': 1352070, 'calculationPrice': 'close', 'change': 4.62, 'changePercent': 0.03183, 'close': 149.76, 'closeSource': 'official', 'closeTime': 1715371202491, 'companyName': 'Agilent Technologies Inc.', 'currency': 'USD', 'delayedPrice': 149.77, 'delayedPriceTime': 1715371193482, 'extendedChange': 0, 'extendedChangePercent': 0, 'extendedPrice': 149.76, 'extendedPriceTime': 1715382000002, 'high': 152.81, 'highSource': '15 minute delayed price', 'highTime': 1715371199990, 'iexAskPrice': 0, 'iexAskSize': 0, 'iexBidPrice': 0, 'iexBidSize': 0, 'iexClose': 149.75, 'iexCloseTime': 1715371194908, 'iexLastUpdated': 1715371194908, 'iexMarketPercent': 0.027424947145877378, 'iexOpen': 145.275, 'iexOpenTime': 1715347801013, 'iexRealtimePrice': 149.75, 'iexRealtimeSize': 2, 'iexVolume': 61617, 'lastTradeTime': 1715371199977, 'latestPrice': 149.76, 'latestSource': 'Close', 'latestTime': 'May 10, 2024'

In [95]:
selected_columns = [[row['symbol'], row['latestPrice'], row['marketCap'], 'N/A'] for row in my_temp_list]

selected_columns = [[row['symbol'], row['latestPrice'], row['marketCap'], 'N/A'] for row in my_temp_list]

df = pd.DataFrame(selected_columns, 
                 columns=my_columns)

df.size

1984

In [131]:
df

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy
0,A,149.76,4.388796e+10,N/A
1,AAL,14.40,9.447701e+09,N/A
2,AAP,75.03,4.472499e+09,N/A
3,AAPL,183.05,2.806904e+12,N/A
4,ABBV,160.75,2.838632e+11,N/A
...,...,...,...,...
491,YUM,137.62,3.875823e+10,N/A
492,ZBH,121.31,2.495691e+10,N/A
493,ZBRA,315.80,1.623825e+10,N/A
494,ZION,44.32,6.544058e+09,N/A


##### Column vs Index difference

In [87]:
import pandas as pd

# Create a DataFrame with custom index and columns
data = [[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]]
df = pd.DataFrame(data, index=['X', 'Y', 'Z'], columns=['Z','S','O'])

# Print DataFrame
print(df)


   Z  S  O
X  1  2  3
Y  4  5  6
Z  7  8  9


In [83]:
import pandas as pd

# Create a DataFrame
data = {[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]}
df = pd.DataFrame(data)


# Print DataFrame with renamed columns
print(df)


TypeError: unhashable type: 'list'

In [130]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy,0
Ticker,NaN,NaN,NaN,NaN,A
Stock Price,NaN,NaN,NaN,NaN,149.76
Market Capitalization,NaN,NaN,NaN,NaN,43887959332
Number of Shares to Buy,NaN,NaN,NaN,NaN,N/A
Ticker,NaN,NaN,NaN,NaN,AAL
Stock Price,NaN,NaN,NaN,NaN,14.4
Market Capitalization,NaN,NaN,NaN,NaN,9447700954
Number of Shares to Buy,NaN,NaN,NaN,NaN,N/A
Ticker,NaN,NaN,NaN,NaN,AAP
Stock Price,NaN,NaN,NaN,NaN,75.03


## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [98]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [100]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_groups

[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object,
 200     GIS
 201      GL
 202     GLW
 203      GM
 204    GOOG
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 100, dtype: object,
 300     MCK
 301     MCO
 302    MDLZ
 303     MDT
 304     MET
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 100, dtype: object,
 400     ROL
 401     ROP
 402    ROST
 403     RSG
 404     RTX
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 100, dtype: object,
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name

In [134]:
df.to_pickle('data.pk1')

## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

We'll do that next.

In [104]:
portfolio_size

'0'

In [119]:
val = None
while not isinstance(val, float):
    try:
        portfolio_size = input('Enter the value of your portfolieo:')
        val = float(portfolio_size)
    except ValueError:
        print('Please enter an float')
    


In [140]:
position_size = val/len(df)
print(position_size)

print(df)

for i in range(0, len(df.index)):
    df.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / df['Stock Price'][i])


number_of_apple_shares = position_size / 500
df

20161.290322580644
    Ticker  Stock Price  Market Capitalization Number of Shares to Buy  \
0        A       149.76           4.388796e+10                     N/A   
1      AAL        14.40           9.447701e+09                     N/A   
2      AAP        75.03           4.472499e+09                     N/A   
3     AAPL       183.05           2.806904e+12                     N/A   
4     ABBV       160.75           2.838632e+11                     N/A   
..     ...          ...                    ...                     ...   
491    YUM       137.62           3.875823e+10                     N/A   
492    ZBH       121.31           2.495691e+10                     N/A   
493   ZBRA       315.80           1.623825e+10                     N/A   
494   ZION        44.32           6.544058e+09                     N/A   
495    ZTS       169.04           7.713213e+10                     N/A   

     Number Of Shares to Buy  
0                      134.0  
1                     1400.0  

,Ticker,Stock Price,Market Capitalization,Number of Shares to Buy,Number Of Shares to Buy
0,A,149.76,4.388796e+10,N/A,134.0
1,AAL,14.40,9.447701e+09,N/A,1400.0
2,AAP,75.03,4.472499e+09,N/A,268.0
3,AAPL,183.05,2.806904e+12,N/A,110.0
4,ABBV,160.75,2.838632e+11,N/A,125.0
...,...,...,...,...,...
491,YUM,137.62,3.875823e+10,N/A,146.0
492,ZBH,121.31,2.495691e+10,N/A,166.0
493,ZBRA,315.80,1.623825e+10,N/A,63.0
494,ZION,44.32,6.544058e+09,N/A,454.0


496

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [157]:
writer = pd.ExcelWriter('recommended trades.xlsx', engine='xlsxwriter')
df.to_excel(writer, 'Recommended Trades', index = False)


In [160]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
    {
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

dollar_format = writer.book.add_format(
    {
        'num_format': '$0.00',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)

integer_format = writer.book.add_format(
    {
        'num_format': '0',
        'font_color': font_color,
        'bg_color': background_color,
        'border': 1
    }
)



### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [161]:
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_template' to the column
                    )

writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)

0

This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [162]:
column_formats = {
    'A': ['Ticker', string_format],
    'B': ['Stock Price', dollar_format],
    'C': ['Market Capitalizaiton', dollar_format],
    'D': ['Number of Shares to Buy', integer_format],
}

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)


writer.close()


## Saving Our Excel Output

Saving our Excel file is very easy: